<a href="https://colab.research.google.com/github/parthbhide/Support-Genie-Python-Developer-Intern/blob/master/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#Read Generated Data
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,start_time,answer_time,resolved_time,abandoned_time,delay,status
0,15/03/2020,18/03/2020,19/03/2020,NaN,3,1
1,14/03/2020,NaN,NaN,15/03/2020,1,0
2,13/03/2020,14/03/2020,15/03/2020,NaN,1,1
3,12/03/2020,NaN,NaN,14/03/2020,2,0
4,11/03/2020,13/03/2020,15/03/2020,NaN,2,1


In [36]:
#Converting start_time to datatime and replacing NaN with 0

df['start_time'] = pd.to_datetime(df['start_time'])
df = df.fillna(0)

In [37]:
def get_quater(dates):
  quater = []
  for x in dates:
    if x.month >= 1 and x.month <= 3:
      quater.append(1)
    if x.month >= 4 and x.month <= 6:
      quater.append(2)
    if x.month >= 7 and x.month <= 9:
      quater.append(3)
    if x.month >= 10 and x.month <= 12:
      quater.append(4)
      
  return quater

**Delay** can depend on many factors such as weekday on which issue was raised, month, financial quater in which issue was raised, location where issue was raised, type of issue raised and so on.

We are here extracting from the available information.

In [65]:
quater = get_quater(df['start_time'])
# for x in df['start_time']:
#   if x.month >= 1 and x.month <= 3:
#     quater.append(1)
#   if x.month >= 4 and x.month <= 6:
#     quater.append(2)
#   if x.month >= 7 and x.month <= 9:
#     quater.append(3)
#   if x.month >= 10 and x.month <= 12:
#     quater.append(4)

df['day'] = [x.day for x in df['start_time']]
df['month'] = [x.month for x in df['start_time']]
df['weekday'] = [x.weekday() for x in df['start_time'] ]
df['year'] = [x.year for x in df['start_time']]
df['quater'] = quater

#Shuffle columns
df = df[['start_time','answer_time','resolved_time','abandoned_time','status','day','month','year','weekday','quater','delay']]
df.head()

,start_time,answer_time,resolved_time,abandoned_time,status,day,month,year,weekday,quater,delay
0,2020-03-15,18/03/2020,19/03/2020,0,1,15,3,2020,6,1,3
1,2020-03-14,0,0,15/03/2020,0,14,3,2020,5,1,1
2,2020-03-13,14/03/2020,15/03/2020,0,1,13,3,2020,4,1,1
3,2020-12-03,0,0,14/03/2020,0,3,12,2020,3,4,2
4,2020-11-03,13/03/2020,15/03/2020,0,1,3,11,2020,1,4,2


In [66]:
#Copying the required data into a new dataframe

train_df = df[['day','month','year','weekday','quater','delay']]
train_df.reset_index(drop=True, inplace =True)
train_df.head()

,day,month,year,weekday,quater,delay
0,15,3,2020,6,1,3
1,14,3,2020,5,1,1
2,13,3,2020,4,1,1
3,3,12,2020,3,4,2
4,3,11,2020,1,4,2


In [67]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [68]:
#Predictions using Random Forest Classifier

model = RandomForestClassifier(n_estimators=120,criterion='entropy',max_depth=2, random_state=42, max_features = None,verbose=True,min_samples_split=5)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
metrics.accuracy_score(y_test,predicted)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    0.0s finished


0.205

In [76]:
def predict_delay(new_issue_date):
  data = pd.DataFrame()
  data['day'] = [new_issue_date.day]
  data['month'] = [new_issue_date.month]
  data['year'] = [new_issue_date.year]
  data['weekday'] = new_issue_date.weekday()
  data['quater'] = get_quater([new_issue_date])
  p = model.predict(data)

  return p[0]

In [78]:
import datetime

issue_date = datetime.datetime.today()

f'Approximately, this issue will be assigned to an agent at : { issue_date + datetime.timedelta(int()) }'

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    0.0s finished


'Approximately, this issue will be assigned to an agent at : 2020-06-27 11:08:07.026046'